# MAGIC + LST combined analysys: event coincidence search

In [1]:
import yaml
import numpy as np
import pandas as pd
import glob
from matplotlib import pyplot as plt
import magicctapipe
print(f'magicctapipe version: {magicctapipe.__version__}')

magicctapipe version: 0.3.0


In [2]:
# --- display all columns --- 
pd.set_option('display.max_columns', None)

# --- customize plt figure ---
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 15
plt.rcParams['grid.linestyle'] = ':'

# --- get the default color cycle ---
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

The script for the coincidence search is <b>lst1_magic_event_coincidence.py<b>

This script searches for coincident events from LST-1 and MAGIC joint observation data offline using their timestamps. It applies the coincidence window to LST-1 events, and checks the coincidence within the time offset region specified in the configuration file. Since the optimal time offset changes depending on the telescope distance along the pointing direction, it is recommended to input one subrun file for LST-1 data, whose observation time is usually around 10 seconds so the change of the distance is negligible. The MAGIC standard stereo analysis discards the events when one of the telescope images cannot survive the cleaning or fail to compute the DL1 parameters. However, it's possible to perform the stereo analysis if LST-1 sees these events. Thus, it checks the coincidence for each telescope combination (i.e., LST1 + M1 and LST1 + M2) and keeps the MAGIC events even if they do not have their MAGIC-stereo counterparts. The MAGIC-stereo events, observed during the LST-1 observation time period but not coincident with any LST-1 events, are also saved in the output file, but they are not yet used for the high level analysis. Unless there is any particular reason, please use the default half width 300 ns for the coincidence window, which is optimized to reduce the accidental coincidence rate as much as possible by keeping the number of actual coincident events. Please note that for the data taken before 12th June 2021, a coincidence peak should be found around the time offset of -3.1 us, which can be explained by the trigger time delays of both systems. For the data taken after that date, however, there is an additional global offset appeared and then the peak is shifted to the time offset of -6.5 us. Thus, it would be needed to tune the offset scan region depending on the date when data were taken. The reason of the shift is under investigation.
    

<b>USAGE<b>:

python lst1_magic_event_coincidence.py

--input-file-lst dl1_LST-1.Run03265.0040.h5

--input-dir-magic dl1/MAGIC

(--output-dir dl1_coincidence)

(--config-file config.yaml)


<b>N.B.<b>:


# 1) Input data

FYI: for the date LST (and its directory system) uses the date of the end of the observation, whereas MAGIC uses that of the start of the observation.

In [3]:
school_data_dir='/fefs/aswg/workspace/2023_joint_analysis_school'

In [4]:
date_lst='20210213'
date_magic='2021_02_14'

In [5]:
dl1_dir=f'{school_data_dir}/event_coincidence/input/DL1'
lst_dl1_dir=dl1_dir+f'/LST1/{date_lst}'
magic_dl1_dir=dl1_dir+f'/MAGIC/{date_magic}'

In [6]:
#input_file_lst = f'{lst_dl1_dir}/dl1_LST-1.Run03631.0039.h5'
input_file_lst = f'{lst_dl1_dir}/dl1_LST-1.Run03636.0039.h5'

Check LST1 dl1 data

In [7]:
from lstchain.io.io import dl1_params_lstcam_key
from lstchain.reco.utils import add_delta_t_key,get_effective_time
from ctapipe.io import read_table

In [8]:
dl1_lst_data=read_table(input_file_lst, dl1_params_lstcam_key)
dl1_lst_data

obs_id,event_id,intensity,log_intensity,x,y,r,phi,length,length_uncertainty,width,width_uncertainty,psi,skewness,kurtosis,time_gradient,intercept,leakage_intensity_width_1,leakage_intensity_width_2,leakage_pixels_width_1,leakage_pixels_width_2,n_pixels,concentration_cog,concentration_core,concentration_pixel,n_islands,alt_tel,az_tel,calibration_id,dragon_time,ucts_time,tib_time,ucts_jump,mc_type,wl,tel_id,tel_pos_x,tel_pos_y,tel_pos_z,trigger_type,ucts_trigger_type,trigger_time,event_type
,,,,m,m,m,rad,deg,deg,deg,deg,rad,,,,,,,,,,,,,,rad,rad,,,,,,,,,m,m,m,,,,
int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float64,float64,int64,float64,float64,float64,int64,float64,float64,int64,float64,int64,int64,bool,int64,float64,int64,float64,float64,float64,uint8,uint8,float64,int64
3636,2067001,203.30835914611816,2.308155235285357,-0.9797356306609176,0.059294361761496515,0.981528261092491,3.0811456056011064,0.17518081187994444,0.005537182698284188,0.07903151957140998,0.0029342396201543874,-0.015303092519049735,0.21925482384007544,1.8124883820724962,1.252771667603426,19.395287030997228,0.08449865,0.26922402,0.001078167115902965,0.0026954177897574125,19,0.2032968605666236,0.23680629413804863,0.08563318645664028,1,0.9908932518453828,4.662985675698991,0,1613259416.8394885,0,0,False,-9999,0.45114255792793195,1,50.0,50.0,16.0,1,1,1613259416.8394885,32
3636,2067002,491.8836064338684,2.6918623485728492,0.2557725194602321,0.22048274015382746,0.33768657127101603,0.7114348771038935,0.35146080076931024,0.010117484695825836,0.21892742642009738,0.005747390111104336,0.2014118647638779,-0.09041293015317466,2.6304323297435523,34.82109780442585,14.80559123704517,0.0,0.0,0.0,0.0,48,0.033537715807910984,0.29748594744645673,0.06091527736732531,3,0.9908932456035989,4.66298567986018,0,1613259416.8396316,0,0,False,-9999,0.62290709501853,1,50.0,50.0,16.0,1,1,1613259416.8396316,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3636,2119999,108.02978372573853,2.033543506639755,0.7171733719093675,0.3756310954862062,0.8095902452920362,0.48247921488864526,0.2851255491662784,0.016705982230751225,0.056943644711571505,0.004868049096731397,-0.16722375604022577,1.1536992522338159,2.483433605647556,-2.5089067785263097,18.84404404038615,0.0,0.13993627,0.0,0.001078167115902965,8,0.1450365069828093,0.6063661123481354,0.41128730148678283,2,0.9900137592055813,4.6635316834022325,0,1613259430.8415422,0,0,False,-9999,0.19971428333264982,1,50.0,50.0,16.0,1,1,1613259430.8415422,32
3636,2120000,85.46019124984741,1.9317638603821894,0.4473936739508037,-0.7906316570944479,0.90843795423304,-1.05585157044516,0.07335652853648726,0.005876983668713766,0.042725082890814024,0.0027700399004860596,1.2483051636514748,-0.8012484558210259,3.194091858876041,-0.037933798252412575,18.838747813160648,0.0,0.0,0.0,0.0,6,0.8829620656244861,0.2672825756732767,0.29844558297260165,1,0.9900137343576642,4.663531696310241,0,1613259430.841912,0,0,False,-9999,0.5824305449454676,1,50.0,50.0,16.0,1,1,1613259430.841912,32


In [9]:
dl1_lst_data =add_delta_t_key(dl1_lst_data)
time=get_effective_time(dl1_lst_data)
print(f'Effective time: {time[0]}\nElapsed time: {time[1]}')

Effective time: 13.585437297821043 s
Elapsed time: 14.002423524856567 s


# 2) Configuration File

In [10]:
config_file = './config_MAGIC_LST.yaml'

with open(config_file, 'rb') as f:
    config = yaml.safe_load(f)

In [11]:
config['event_coincidence']

{'timestamp_type_lst': 'dragon_time',
 'window_half_width': '300 ns',
 'time_offset': {'start': '-10 us', 'stop': '0 us'}}

1) <b> 'timestamp_type_lst'<b> :
- "dragon_time"
- "tib_time"
- "ucts_time"
  
2) <b>'window_half_width'<b>:

    the algorithm searches for a coincident event in MAGIC dataset, with a timestmap included within half of the coincidence window from LST-1 event timestamp

<img src="coinc_window.png" alt="Alternative text" />


3) <b>'time_offset'<b>:

    to find the final offset value to use for the ocincidence search, the algorithm scan different offset values within a given range (start and stop)


# 3) Running the script

In [12]:
output_dir = f'{school_data_dir}/event_coincidence/output'
# If you are working at the IT container, create your own loca loutput directory
#output_dir = './coinc/output'
!mkdir -p $output_dir

Execute script from terminal:

In [ ]:
!lst1_magic_event_coincidence\
--input-file-lst $input_file_lst\
--input-dir-magic $magic_dl1_dir\
--output-dir $output_dir\
--config-file $config_file


Input LST-1 DL1 data file: /fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/LST1/20210213/dl1_LST-1.Run03636.0039.h5
LST-1: 48108 events

Input MAGIC directory: /fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14

The following DL1 data files are found:
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M1.Run05094487.001.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M1.Run05094487.002.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M1.Run05094487.003.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M1.Run05094487.004.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M1.Run05094487.005.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/

/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M2.Run05094488.011.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M2.Run05094488.012.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M2.Run05094488.013.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M2.Run05094488.014.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M2.Run05094488.015.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M2.Run05094488.016.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M2.Run05094488.017.h5
/fefs/aswg/workspace/2023_joint_analysis_school/event_coincidence/input/DL1/MAGIC/2021_02_14/dl1_M2.Run05094488.018.h5
/fefs/aswg/workspace/2023_joint_analysis_school/

Import function on a notebook:

In [ ]:
from magicctapipe.scripts.lst1_magic import event_coincidence

In [ ]:
#event_coincidence(input_file_lst=input_file_lst, input_dir_magic=magic_dl1_dir, output_dir=output_dir, config=config)

# 4) Check the output data

The output file contains 3 tables :
1) coincidence

    - feature
    
    it contains:
        - the subrun average timing and poinying information for both MAGIC and LST
        - the average offset, coincidence window and number of found coincident events
    for the  2 'coincidence_id' types
        -12 = M1 - LST
        -13 = M2 - LST
        
    - profile
    
    it contains the number identified coincident events for the 2 'coincidence_id' types, for all the tried time offsets
    
    
2) configuration/instrument

      it contains a series of camera and instrumental information for the MAGIC telescopes


3) events:

    - parameters:
    
    it contains all the coincident events information:
        - LST run ID
        - MAGIC run ID
        - the combnation ('combo_type') of telescopes that saw the events:
            - 0 --> M1 + M2 (only)
            - 1 --> LST-1 + M1
            - 2 --> LST-1 + M2
            - 3 --> LST-1 + M1 + M2
        - the multuplicty of the event --> total number of telescopes that saw the event (either 2 or 3)
        - the event single-telescope parameters for all the telescopes that saw the event
            

In [ ]:
tel_combos = {12: "LST1 - M1", 13: "LST1 - M2"}

In [ ]:
output_file=f'{output_dir}/dl1_LST-1_MAGIC.Run03636.0039.h5'

# 4.1) Check the coincident event table

In [ ]:
data = pd.read_hdf(output_file, key='events/parameters')
data.set_index(['obs_id', 'event_id', 'tel_id'], inplace=True)
data.sort_index(inplace=True)

In [ ]:
data.head(10)

In [ ]:
data.columns

<b> Check the intensity correlation between LST-1 and MAGIC <b>

In [ ]:
plt.figure(figsize=(20, 7))
grid = (1, 2)

df = data.query('multiplicity == 3')

for i_grid, (tel_name, tel_id) in enumerate(zip(['MAGIC-I', 'MAGIC-II'], [2, 3])):
    
    plt.subplot2grid(grid, (0, i_grid))
    plt.title(f'LST-1 vs {tel_name}')
    plt.xlabel(f'{tel_name} intensity [phe]')
    plt.ylabel('LST-1 intensity [phe]')
    plt.loglog()
    
    plt.hist2d(
        df.query(f'tel_id == {tel_id}')['intensity'].to_numpy(),
        df.query(f'tel_id == 1')['intensity'].to_numpy(),
        bins=np.logspace(1, 5, 41),norm='log'
    )
    
    plt.plot((1e1, 1e5), (1e1, 1e5), linestyle='--', color='black')
    
    plt.colorbar()
    
    plt.grid()

# 4.2) Check the coincidence distribution

In [ ]:
profile = pd.read_hdf(output_file, key='coincidence/profile')
profile

In [ ]:
features = pd.read_hdf(output_file, key='coincidence/feature')
features

In [ ]:
plt.figure(figsize=(12, 7))
plt.xlabel('Time offset [us]')#,fontsize=24,fontweight='bold'),
plt.ylabel('Number of coincident events')#,fontsize=24,fontweight='bold'),
plot_xlim=[-5e6,-1e6]
sec2us = 1e6
ns2us = 1e3
for color,tel_combo in zip(['red','blue'],tel_combos):
    
    combo_name=tel_combos[tel_combo]
    
    plt.plot(
        profile['time_offset'].values * sec2us, profile[f'n_coincidence_tel{tel_combo}'].values, 
        linestyle="None", marker='o', color=color
    )
    
    plt.step(
        profile['time_offset'].values * sec2us, profile[f'n_coincidence_tel{tel_combo}'].values, 
        where='post', label=combo_name,color=color
    )
    
    offset_usec_avg = features.query(f'coincidence_id == {tel_combo}')['average_offset'].values * sec2us
    window_half_width = features.query(f'coincidence_id == {tel_combo}')['window_half_width'].values * ns2us
    
    plt.plot(
        np.repeat(offset_usec_avg, 2), (0, plt.ylim()[1]+10), linestyle='--', alpha=0.8, color=color
    )
    a=plt.ylim()[1]
    plt.axvspan(offset_usec_avg-window_half_width,offset_usec_avg+window_half_width,alpha=0.1, color=color)
    #plt.arrow(offset_usec_avg+window_half_width,a+10,1,0,head_width=30, head_length=20000, linewidth=4, color=color,length_includes_head=True)
    #plt.arrow(offset_usec_avg+window_half_width,a+10,-window_half_width,0,head_width=30, head_length=20000, linewidth=4, color=color,length_includes_head=True,label='coincidence window')

text=f'Offset region = 2*coincidece window'
plt.text(plot_xlim[0]-plot_xlim[0]/500,plt.ylim()[1]-300,s=text,fontsize=15)
plt.grid()
plt.xlim(plot_xlim[0],plot_xlim[1])
#plt.xticks(fontsize=24,fontweight='bold')
#plt.yticks(fontsize=24,fontweight='bold')
plt.legend()

# 5) Run-wise checks

For for file-size reasons, for this part we will load already produced 'dl1_coincidence' sub-run files 

# 5.1) Load all the subrun input files

In [ ]:
input_dl1_coinc_dir=f'{dl1_dir}/DL1_coinc'

In [ ]:
print("Loading the input files...")
#input_files=glob.glob(f'{school_data_dir}/stereo_reconstruction/input/DL1_coinc/{date_lst}/*.h5')
input_files=glob.glob(f'{input_dl1_coinc_dir}/{date_lst}/*3636*.h5')
input_files.sort()
n_files=len(input_files)
data_list=[]
for i_file, input_file in enumerate(input_files):

    if i_file % 10 == 0:
        print(f"{i_file}/{n_files}")

    # Load the coincidence features
    df = pd.read_hdf(input_file, key="coincidence/feature")
    data_list.append(df)

print(f"{n_files}/{n_files}")

# Combine the data frames
features = pd.concat(data_list)

# Show the data frame
features.head()

# 5.2) Check the average offset

In [ ]:
plt.figure(figsize=(20, 7))
plt.xlabel("Unix time [sec]")
plt.ylabel("Average offset [us]")
plt.grid()

# Loop over every coincidence ID
for tel_combo in tel_combos:
    
    combo_name=tel_combos[tel_combo]

    df_features = features.query(f"coincidence_id == {tel_combo}")

    # Plot the average offset
    plt.plot(
        df_features["unix_time"],
        df_features["average_offset"],
        marker="o",
        label=combo_name,
    )

plt.legend()

# 5.2) Check the number of coincident events

In [ ]:
plt.figure(figsize=(20, 7))
plt.xlabel("Unix time [sec]")
plt.ylabel("Number of coincident events")
plt.grid()

# Loop over every coincidence ID
for tel_combo in tel_combos:
    
    combo_name=tel_combos[tel_combo]

    df_features = features.query(f"coincidence_id == {tel_combo}")

    # Plot the number of coincident events
    plt.plot(
        df_features["unix_time"],
        df_features["n_coincidence"],
        marker="o",
        label=combo_name,
    )

plt.legend()

# 5.3) Check the fraction of coincidence

In [ ]:
plt.figure(figsize=(20, 7))
plt.xlabel("Unix time [sec]")
plt.ylabel("Fraction (= N$_\mathrm{coinc}$ / N$_\mathrm{magic}$)")
plt.ylim(0, 1)
plt.grid()

# Loop over every coincidence ID
for tel_combo in tel_combos:
    
    combo_name=tel_combos[tel_combo]

    df_features = features.query(f"coincidence_id == {tel_combo}")
    fractions = df_features["n_coincidence"] / df_features["n_events_magic"]

    # Plot the fraction of coincidence
    plt.plot(df_features["unix_time"], fractions, marker="o", label=combo_name)

plt.legend()

# 6) Check cocindences in multiple runs

<b> N.B: For this part the data are not stored in the school directory: no need to download so many files, it's just to show you the plots. <b>
    
<font color='red'> <b> DON'T EXECUTE THE FOLLOWING CELLS <b> </font>

Two main things can be noticed:

    - The drop of coincident events at the end of each run due to the wobble change
    - The change of average time offset and over all numerber of coincidencent events with time, due to the change of source position.

In [ ]:
run_list=[3634,3635,3636]
multiple_run_dir='/fefs/aswg/workspace/giorgio.pirola/Software_School_2023/data/real/joint/DL1/20210213'

In [ ]:
data_list_2runs=[]
for run_number in run_list:
    print(f"Loading the input files for run {run_number}...")
    input_files=glob.glob(f'{multiple_run_dir}/*{run_number}*.h5')
    input_files.sort()
    n_files=len(input_files)

    for i_file, input_file in enumerate(input_files):

        if i_file % 10 == 0:
            print(f"{i_file}/{n_files}")

        # Load the coincidence features
        df = pd.read_hdf(input_file, key="coincidence/feature")
        df['run_number']=run_number
        data_list_2runs.append(df)
    
    print(f"{n_files}/{n_files}")

# Combine the data frames
features_2runs = pd.concat(data_list_2runs)
features_2runs=features_2runs.set_index('run_number')
# Show the data frame
features_2runs.head()

In [ ]:
plt.figure(figsize=(20, 14))
plt.xlabel("Unix time [sec]")
plt.ylabel("Average offset [us]")
grid = (2, 1)

for i_grid,tel_combo in enumerate(tel_combos):
    ax = plt.gca()
    combo_name=tel_combos[tel_combo]
    plt.subplot2grid(grid, (i_grid,0))
    
    
    plt.title(f'{combo_name}')
    for run_number in run_list:
        color = next(ax._get_lines.prop_cycler)['color']
        features_run=features_2runs.iloc[features_2runs.index.get_level_values('run_number') == run_number]
        df_features = features_run.query(f"coincidence_id == {tel_combo}")
        
        plt.plot(
            df_features["unix_time"],
            df_features["average_offset"],
            marker="o",
            label=run_number,
        )
    plt.grid()
    plt.legend()

In [ ]:
plt.figure(figsize=(20, 14))

plt.xlabel("Unix time [sec]")
plt.ylabel("Number of coincident events")

# Loop over every coincidence ID
grid = (2, 1)

for i_grid,tel_combo in enumerate(tel_combos):
    ax = plt.gca()
    combo_name=tel_combos[tel_combo]
    plt.subplot2grid(grid, (i_grid,0))
    
    
    plt.title(f'{combo_name}')
    for run_number in run_list:
        color = next(ax._get_lines.prop_cycler)['color']
        features_run=features_2runs.iloc[features_2runs.index.get_level_values('run_number') == run_number]
        df_features = features_run.query(f"coincidence_id == {tel_combo}")
        
        # Plot the number of coincident events
        plt.plot(
            df_features["unix_time"],
            df_features["n_coincidence"],
            marker="o",
            label=run_number,
            color=color
        )
    plt.grid()
    plt.legend()

In [ ]:
plt.figure(figsize=(20, 14))

plt.xlabel("Unix time [sec]")
plt.ylabel("Number of coincident events")

# Loop over every coincidence ID
grid = (2, 1)

for i_grid,tel_combo in enumerate(tel_combos):
    ax = plt.gca()
    combo_name=tel_combos[tel_combo]
    plt.subplot2grid(grid, (i_grid,0))
    
    
    plt.title(f'{combo_name}')
    for run_number in run_list:
        color = next(ax._get_lines.prop_cycler)['color']
        features_run=features_2runs.iloc[features_2runs.index.get_level_values('run_number') == run_number]
        df_features = features_run.query(f"coincidence_id == {tel_combo}")
        
        fractions = df_features["n_coincidence"] / df_features["n_events_magic"]

    # Plot the fraction of coincidence
        plt.plot(df_features["unix_time"], fractions, marker="o", label=run_number)
    
    plt.grid()
    plt.legend()

In [ ]:
k=0
plt.figure(figsize=(21, 12))
colors=['red','blue','green']
for tel_combo in (tel_combos):
    
    combo_name=tel_combos[tel_combo]
    
    for j,run_number in enumerate(run_list):
        color=colors[j]
        features_run=features_2runs.iloc[features_2runs.index.get_level_values('run_number') == run_number]
        df_features = features_run.query(f"coincidence_id == {tel_combo}")
        a=231+k
        
        plt.subplot(a)
        plt.title(f'{combo_name}')
        plt.plot(
            df_features["unix_time"],
            np.rad2deg(df_features["pointing_alt_lst"]),
            marker="",
            label=run_number,
            color=color

        )
        plt.plot(
            df_features["unix_time"],
            np.rad2deg(df_features["pointing_alt_magic"]),
            marker="o",
            markersize=20,
            color=color,
            alpha=0.01
        )
        plt.xlabel("Unix time [sec]")
        plt.ylabel("Altitude [deg] ")
        plt.grid()
        plt.legend()
        
        
        plt.subplot(a+1)
        plt.title(f'{combo_name}')
        plt.plot(
            df_features["unix_time"],
            np.rad2deg(df_features["pointing_az_lst"]),
            marker="",
            color=color,
            label=run_number,
    
        )
        plt.plot(
            df_features["unix_time"],
            np.rad2deg(df_features["pointing_az_magic"]),
            marker="o",
            markersize=20,
            color=color,
            alpha=0.01
        )
        plt.xlabel("Unix time [sec]")
        plt.ylabel("Altitude [deg] ")
        plt.grid()
        plt.legend()
        
        
        plt.subplot(a+2)
        plt.title(f'{combo_name}')
        plt.plot(
            np.rad2deg(df_features["pointing_az_lst"]),
            np.rad2deg(df_features["pointing_alt_lst"]),
            marker="",
            label=run_number,
            color=color
        )
        plt.plot(
            np.rad2deg(df_features["pointing_az_magic"]),
            np.rad2deg(df_features["pointing_alt_magic"]),
            marker="o",
            markersize=20,
            color=color,
            alpha=0.01
        )
        plt.xlabel("Azimuth [deg] ")
        plt.ylabel("Altitude [deg] ")
        plt.grid()
        plt.legend()
    k=3

In [ ]:
!rm -r $output_dir